
# データの前処理

:ラベル: `sec_pandas`

これまでのところ、既製のテンソルで到着した合成データを扱ってきました。ただし、ディープラーニングを実際に適用するには、任意の形式で保存された乱雑なデータを抽出し、ニーズに合わせて前処理する必要があります。幸いなことに、 *pandas*[ライブラリは](https://pandas.pydata.org/)面倒な作業の多くを行うことができます。このセクションは、適切な*pandas*[チュートリアル](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)の代わりにはなりませんが、最も一般的なルーチンのいくつかについての短期集中コースを提供します。

## データセットの読み取り

カンマ区切り値 (CSV) ファイルは、表形式 (スプレッドシートのような) データを保存するために広く普及しています。ここで、各行は 1 つのレコードに対応し、複数の (カンマ区切り) フィールドで構成されます (例: 「Albert Einstein、1879 年 3 月 14 日、ウルム、連邦工科大学、重力物理学の分野での成果」)。 `pandas`で CSV ファイルをロードする方法を示すために、 `../data/house_tiny.csv` house_tiny.csv を作成します (**以下に CSV ファイルを作成します**)。このファイルは住宅のデータセットを表します。各行は個別の住宅に対応し、列は部屋の数 ( `NumRooms` )、屋根のタイプ ( `RoofType` )、および価格 ( `Price` ) に対応します。


In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')


次に、 `pandas`インポートし、 `read_csv`を使用してデータセットをロードしましょう。


In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000



## データの準備

教師あり学習では、*入力*値のセットが与えられた場合に、指定された*目標*値を予測するようにモデルをトレーニングします。データセットを処理する最初のステップは、入力値とターゲット値に対応する列を分離することです。名前または整数位置ベースのインデックス作成 ( `iloc` ) によって列を選択できます。

 `pandas`値`NA`を持つすべての CSV エントリを特別な`NaN` (*数値ではない*) 値に置き換えたことに気づいたかもしれません。これは、エントリが空の場合 (例: "3,,,270000") にも発生する可能性があります。これらは*欠損値*と呼ばれ、データ サイエンスの「トコジラミ」であり、キャリアを通じて直面することになる永続的な脅威です。コンテキストに応じて、欠損値は*補完*または*削除*によって処理される場合があります。代入では欠損値がその値の推定値で置き換えられますが、削除では欠損値を含む行または列が単に破棄されます。

ここでは、一般的な代入ヒューリスティックをいくつか紹介します。 [**カテゴリ入力フィールドの場合、 `NaN`カテゴリとして扱うことができます。** ] `RoofType`列は値`Slate`と`NaN`を取るため、 `pandas`この列を`RoofType_Slate`と`RoofType_nan` 2 つの列に変換できます。屋根タイプが`Slate`である行は、 `RoofType_Slate`と`RoofType_nan`の値をそれぞれ 1 と 0 に設定します。 `RoofType`値が欠落している行については、その逆が当てはまります。


In [3]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       NaN               0             1
1       2.0               0             1
2       4.0               1             0
3       NaN               0             1



数値が欠落している場合、一般的なヒューリスティックの 1 つは、[ **`NaN`エントリを対応する列の平均値で置き換える**] ことです。


In [4]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0               0             1
1       2.0               0             1
2       4.0               1             0
3       3.0               0             1



## Tensor 形式への変換

[ **`inputs`と`targets`のすべてのエントリが数値であるため、それらを tensor にロードすることができます**] (:numref: `sec_ndarray`を思い出してください)。


In [5]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(targets.values)
X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))


## 議論

データ列を分割し、欠落している変数を代入し、 `pandas`データをテンソルにロードする方法がわかりました。 :numref: `sec_kaggle_house`では、さらにデータ処理スキルを学びます。この短期集中コースでは物事はシンプルに保たれましたが、データ処理は複雑になる可能性があります。たとえば、データセットは単一の CSV ファイルではなく、リレーショナル データベースから抽出された複数のファイルに分散している場合があります。たとえば、電子商取引アプリケーションでは、顧客の住所が 1 つのテーブルに存在し、購入データが別のテーブルに存在する場合があります。さらに、実務者は、カテゴリや数値を超えた無数のデータ タイプに直面します。他のデータ タイプには、テキスト文字列、画像、音声データ、点群などがあります。多くの場合、データ処理が機械学習パイプラインの最大のボトルネックになるのを防ぐために、高度なツールと効率的なアルゴリズムが必要になります。これらの問題は、コンピューター ビジョンや自然言語処理に到達すると発生します。最後に、データの品質に注意を払う必要があります。現実世界のデータセットは、外れ値、センサーからの誤った測定値、記録エラーに悩まされることが多く、データをモデルに入力する前に対処する必要があります。 [seaborn](https://seaborn.pydata.org/) 、 [Bokeh](https://docs.bokeh.org/) 、 [matplotlib](https://matplotlib.org/)などのデータ視覚化ツールは、データを手動で検査し、対処する必要がある問題についての直感を養うのに役立ちます。

## 演習
1. データセット (例: Abalone) を[UCI Machine Learning リポジトリ](https://archive.ics.uci.edu/ml/datasets.php)からロードして、そのプロパティを調べてみます。それらのどの部分に欠損値がありますか?変数のどの部分が数値、カテゴリ、またはテキストですか?
1. 列番号ではなく名前でデータ列のインデックス付けと選択を試してください。[インデックス作成](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)に関するパンダのドキュメントには、これを行う方法の詳細が記載されています。
1. この方法でどれくらいの大きさのデータセットを読み込めると思いますか?どのような制限があるでしょうか?ヒント: データ、表現、処理、およびメモリ フットプリントの読み取りにかかる時間を考慮してください。ラップトップでこれを試してみてください。サーバー上で試してみると何が変わるでしょうか?
1. 非常に多くのカテゴリを持つデータをどのように処理しますか?カテゴリ ラベルがすべて一意である場合はどうなるでしょうか?後者も含めるべきでしょうか?
1. パンダの代わりに何が考えられますか? [NumPy テンソルをファイルからロードしてみて](https://numpy.org/doc/stable/reference/generated/numpy.load.html)はどうでしょうか? Python イメージング ライブラリである[Pillow](https://python-pillow.org/)をチェックしてください。



[ディスカッション](https://discuss.d2l.ai/t/29)
